# Traveling Salesman Problem (TSP)

**Objectives**

- Introduce students to a real world problem solved by OR practitioners
- Demonstrate the use of heuristics to obtain good solutions to optimization problems
- Give students an appreciation of the difficulty of solving optimization problems exactly

**Reading:** Read Handout 2 on the traveling salesman problem.

**Brief description:** Finding an optimal solution to a Traveling Salesman Problem, and proving that it is, in fact, an optimal solution, is a difficult task. In practice, when a feasible solution to a difficult problem needs to be provided quickly, one often resorts to using heuristics, i.e., procedures for generating feasible solutions, or improving existing ones, that can be executed quickly and, hopefully, produce a pretty good result. In this lab we will consider several such heuristic procedures for the TSP.

*TSP VLSI instances adapted from [TSPLIB](http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/) and [Bonn Institute](http://www.math.uwaterloo.ca/tsp/vlsi/index.html)*

<font color='blue'> <b>Solutions are shown blue.</b> </font> <br>
<font color='red'> <b>Instuctor comments are shown in red.</b> </font>

<font color='red'>A tool we might want to use: [TSP DIY](https://www.math.uwaterloo.ca/tsp/app/diy.html)  </font>

## Jupyter Notebook Introduction

The labs for this course will be done through Jupyter Notebooks. Each lab will be distributed to you via a `.zip` file that contains a Jupyter Notebook (which has file extension `.ipynb`) and any other files necessary to run that lab. This may include supplementary python code, images, or data.

In this first lab, we will begin with a breif introduction to Jupyter Notebooks. The notebook consists of cells of two main types: text (Markdown) and Code. This notebook is comprised only of text cells up until this point. The text is written in a lightweight markup language called Markdown. It allows you to do things like make tables and mathematical equations. Right now, you are seeing the Markdown text after it is already compiled. Double-click the text `CLICK HERE` to see the Markdown before it is compiled.

`CLICK HERE` Now that you have double-clicked, you should be able to type in this cell. Try it out:



To compile this text, press either CTRL+Enter to compile or Shift+Enter to compile and move to the next cell.

Here are some other cool things you can do in Markdown!

- **Bold** and *italics*
- Equations: 
    - $3x_1 + 4x_2 = 10$ 
    - $\frac{2}{3} + \frac{2}{3} = \frac{4}{3}$
    - $2^x = 4$
- Tables:

|   | 1 | 2 | 3 | 4 |
|---|---|---|---|---|
| a |   |   |   |   | 
| b |   |   |   |   | 

And now to the most important cell of a Jupyter Notebook: the Code cell! We can use a code cell to write and run Python code. Let's look at an example. Like compiling the Markdown cell, to run the code in a Code cell we can use either CTRL+Enter to compile or Shift+Enter to compile and move to the next cell. 

In [47]:
# This is a code cell
# We use # to make a comment in a code cell
a = 7

The cell  above creates a variable called `a` and sets its value to 7. This variable and its value carries over to the other cells of the notebook. To the left of a code cell, we can see a number in brackets if the cell has been run. If it has not been run, there is no number. The number indicates the order in which the cells are run. If a variable is on the last line of code cell, its value will be output under the cell.

In [48]:
a

7

Here are a few examples of some different operations in python.

In [49]:
# addition and multiplication
b = 3
c = a + b
print(c)
d = a * b
print(d)

# lists
array = [1,2,3]
print(array[1])

array # recall, since this is on the last line, its value will be printed

10
21
2


[1, 2, 3]

As we do more and more in Jupyter Notebooks throughout the rest of these labs, we will introduce new tools and functionality. **NOTE:** It is important to mention that this course does not require any previous programming experience nor is it a programming course. The programming in this course will be limited to small changes to code and writing mathematical models. 

On to the lab!

## Part I: Solving TSP Manually

In [50]:
# Imports -- make sure you run this cell!
# This cell contains multiple import statements. Each import statement gives the notebook access to pre-bundled
# python code that serves some functionality. The first import statement imports all of the python code in the
# file tsp.py. You can open this file and examine it if you so choose!

from tsp import *
import vinal as vl
import pandas as pd
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In lectute, you learned about the traveling salesman problem (TSP). The input to this problem is a set of cities along with the distances between them. Our goal is to find a path that visits every city starting and ending at the same city that minimizes the distance traveled. We call a path like this a tour. 

Consider the following TSP problem consisting of 23 US cities. Fun fact: this instance comes from the Beyoncé *On the Run II Tour*.

In [51]:
# This cell imports a CSV file called us_cities_23.csv. We will often import CSV files.
# The data from the CSV file is put in the variable nodes.
# Using display(), we can see the contents of this file: each city (node) and its (x,y) position.
nodes = pd.read_csv('data/us_cities_23.csv', index_col=0)
display(nodes)

,x,y
0,1800,964
1,2058,878
2,2104,936
3,2146,988
4,2244,1090
5,1340,1116
6,1566,970
7,1758,1018
8,1776,898
9,1942,1072


We can use the function `vl.create_network()` to create an graph from this CSV file of nodes.

In [52]:
# The help command returns information about what parameters this function takes
help(vl.create_network)

Help on function create_network in module vinal.build:

create_network(nodes:pandas.core.frame.DataFrame, edges:pandas.core.frame.DataFrame=None, directed:bool=False, manhattan:bool=False, **kw) -> networkx.classes.graph.Graph
    Return networkx graph derived from the list of nodes/edges.
    
    If no edges are given, defaults to generating all edges with
    weight equivalent to the euclidean distance between the nodes.
    
    Args:
        nodes (pd.DataFrame): Dataframe of nodes with positional columns (x,y).
        edges (pd.DataFrame): Dataframe of edges (u,v) with weight w.
        directed (bool): True iff graph is directed (defaults to False).
        manhattan (bool): {True: manhattan distance, False : euclidian}.
    
    Returns:
        Union[nx.Graph, nx.DiGraph]: Either an undirected or directed graph.



Since we only have a dataframe of nodes, this function will generate the distances between them. The parameter `manhattan` indicates how the distances should be computed.  If manhattan is true, the manhatten distance is computed. Otherwise, the euclidean distance is computed. For a city located at $(a,b)$ and a city at $(c,d)$, the manhattan distance is $|c-a| + |d-b|$ (horizontal distance plus the vertical distance). The euclidean distance is $\sqrt{(c-a)^2 + (d-b)^2}$ (striaght-line distance).

In [53]:
# create the graph using the euclidean distances
G = vl.create_network(nodes, manhattan=False)

Now we can find a tour manually. Running the cell below will generate a visual of the 23 cities. Click on the cities one at a time to create a tour. Clicking on the last node will automatically complete the tour. In the lower-left, you will see the cost of the tour update as you create it. In the lower-right, you will see the tour.

In [54]:
# Reminder: Click this cell and press either CTRL+Enter or Shift+Enter to run the cell
show(vl.create_tour_plot(G, width=600, height=375, image='images/us.png'))

**Q1:** What was the smallest tour cost you found?

**A:** <font color='blue'> No less than ???</font>

## Part II: TSP Heuristics

In this part of the lab, we will consider 4 different TSP heuristics. A heuristic aims to find a good feasible solution to a problem although it is not guaranteed to be optimal. Before we move on, we will abstract the TSP to finding an optimal tour on a set of *nodes* rather than cities. These nodes could represent anything. 

- **Random Neighbor:** Start at some node. Randomly select one of the nodes which has not been visited to visit next. Continue doing so until all nodes have been visited. Return to the start.
- **Nearest Neighbor:** Start at some node. Visit the closest unvisited node next (if there are multiple closest nodes, choose one randomly). Continue doing so until all nodes have been visited. Return to the start.
- **Nearest Insertion:** Start with a “tour” on two of the nodes (e.g., the closest pair of nodes). Find the closest unvisited node to any node currently in tour. Insert the node into the tour at the best place (if there are multiple closest nodes, choose one to add randomly).
- **Furthest Insertion:** Start with a “tour” on two of the nodes (e.g., the closest pair of nodes). Find the node whose smallest distance to a node already in the tour is maximized. Insert the node into the tour at the best place (if there are multiple furthest nodes, choose one to add randomly).

**Q2:** Which heuristic to you expect to perform the best? Which do you expect to perform the worst?

**A:** <font color='blue'> Will vary. Maybe random neighbor worst and nearest insertion best.</font>

To compare the heuristics, we will use a simple 6x8 grid of nodes. The cell below creates this instance. Note that we will initally use the manhattan distance.

In [55]:
# nodes is the list of nodes and their position and G is the distance matrix
G = vl.grid_instance(6, 8, manhattan=True)

Let's use random neighbor (a terrible heuristic) to get a baseline for the length of a tour. We will use a function called `plot_tsp_heuristic` to see the mechanics of the algorithm. Run the cell and use the `Previous` and `Next` buttons to move through the iterations of the algorithm. The tour cost will update in the bottom-left. `done.` will appear in the bottom-right when the heuristic has finished.  

In [56]:
show(vl.tsp_heuristic_plot(G, 'random_neighbor', i=0))

To view the complete tour right away, we can use the function `random_neighbor` to run the random neighbor heuristic and the function `plot_tour` to plot the tour and its cost in the lower-left.

In [57]:
tour = vl.random_neighbor(G)
# tour is an ordered list of the nodes starting and ending at the same node
print(tour)
show(vl.tour_plot(G, tour))

[0, 26, 29, 13, 40, 1, 20, 5, 36, 46, 31, 32, 45, 24, 12, 43, 28, 6, 8, 35, 16, 21, 3, 7, 33, 15, 23, 22, 9, 47, 38, 44, 37, 27, 2, 14, 42, 25, 34, 18, 10, 41, 39, 19, 11, 30, 4, 17, 0]


**Q3:** Does this look like a good tour to you? Run it a few times and see what the average tour cost is.

**A:** <font color='blue'>No. Average around 230.</font>

Now, let's look at the nearest neighbor heuristic.

In [58]:
show(vl.tsp_heuristic_plot(G, 'nearest_neighbor', i=0))

**Q4:** As you iterate through, examine the "choices" made by the algorithm at each step. What does it do well? What does it do poorly?

**A:** <font color='blue'>It does better than random neighbor because it often moves to a node close to the one it is currently on. However, it can essentially box itself out of certain regions of the graph. In the end, it often has to make lengthy jumps to get nodes it missed along the way. </font>

**Q5:** Run this a few times. Do you get the same tour every time? Why or why not?

**A:** <font color='blue'>No, because if there are multiple choices for the closest node, one is chosen randomly.</font>

Now, let's look at the nearest insertion heuristic.

In [59]:
show(vl.tsp_heuristic_plot(G, 'nearest_insertion', initial_tour=[0,1,0]))

**Q6:** Run this a few times. How does is compare to the previous heuristics?

**A:** <font color='blue'> This is the best heuristic yet. By starting with a small tour and expanding it, the boxing out issue nearest neighbor experienced is reduced. </font>

Now, let's look at the furthest insertion heuristic.

In [60]:
show(vl.tsp_heuristic_plot(G, 'furthest_insertion', initial_tour=[0,47,0]))

**Q7:** Run this a few times. How does is compare to the previous heuristics?

**A:** <font color='blue'> This heuristic is comparable to nearest insertion although there may be certain circumstanes where one would be more likely to outperform the other. </font>

To compare the heuristics further, lets run each on the 6x8 grid say, 250 times.

**Q8:** Now that you have seen each heuristic, which do you think will do the best and which will do the worst?

**A:** <font color='blue'> Random neighbor will do the worst and either nearest or furthest insertion will do the best. </font>

In [61]:
n = 250
random_neighbor_total = 0
nearest_neighbor_total = 0
nearest_insertion_total = 0
furthest_insertion_total = 0
for i in range(n):
    random_neighbor_total += vl.tour_cost(G, vl.random_neighbor(G))
    nearest_neighbor_total += vl.tour_cost(G, vl.nearest_neighbor(G))
    nearest_insertion_total += vl.tour_cost(G, vl.nearest_insertion(G))
    furthest_insertion_total += vl.tour_cost(G, vl.furthest_insertion(G))
print("Heuristic Averages:")
print("Random Neighbor: %s" % (random_neighbor_total / n))
print("Nearest Neighbor: %s" % (nearest_neighbor_total / n))
print("Nearest Insertion: %s" % (nearest_insertion_total / n))
print("Furthest Insertion: %s" % (furthest_insertion_total / n))

Heuristic Averages:
Random Neighbor: 224.344
Nearest Neighbor: 63.536
Nearest Insertion: 59.984
Furthest Insertion: 57.968


**Q9:** What were the results? Was this what you expected?

**A:** <font color='blue'> Random neighbor did significantly worse. Furthest insertion did the best with nearest insertion and nearest neighbor not to much worse. This was what I expected. </font>

Let's look at 9x9 grid using the euclidian distance now. Run each of the cells below to see each heuristic executed on the new instance.

In [62]:
G = vl.grid_instance(9, 9, manhattan=False)

In [63]:
tour = vl.random_neighbor(G)
show(vl.tour_plot(G, tour))

In [64]:
show(vl.tsp_heuristic_plot(G, 'nearest_neighbor', i=0))

In [65]:
show(vl.tsp_heuristic_plot(G, 'nearest_insertion', initial_tour=[0,1,0]))

In [66]:
show(vl.tsp_heuristic_plot(G, 'furthest_insertion', initial_tour=[0,80,0]))

**Q10:** How did the results compare to the 6x8 grid using manhattan distances?

**A:** <font color='blue'> Similar. </font>

Again, let's run each heuristic numerous times and see how they compare.

In [67]:
n = 100
random_neighbor_total = 0
nearest_neighbor_total = 0
nearest_insertion_total = 0
furthest_insertion_total = 0
for i in range(n):
    random_neighbor_total += vl.tour_cost(G, vl.random_neighbor(G))
    nearest_neighbor_total += vl.tour_cost(G, vl.nearest_neighbor(G))
    nearest_insertion_total += vl.tour_cost(G, vl.nearest_insertion(G))
    furthest_insertion_total += vl.tour_cost(G, vl.furthest_insertion(G))
print("Heuristic Averages:")
print("Random Neighbor: %s" % (random_neighbor_total / n))
print("Nearest Neighbor: %s" % (nearest_neighbor_total / n))
print("Nearest Insertion: %s" % (nearest_insertion_total / n))
print("Furthest Insertion: %s" % (furthest_insertion_total / n))

Heuristic Averages:
Random Neighbor: 381.8990192722607
Nearest Neighbor: 100.93480670792033
Nearest Insertion: 87.48877340603538
Furthest Insertion: 84.87027322002871


**Q11:** How did the results compare to the 6x8 grid using manhattan distances?

**A:** <font color='blue'> Similar. </font>

## Part III: Improving Tours: 2-OPT

In **Part II**, we used heuristics to create TSP tours. However, we can also use heuristics to try and improve tours we have already found. First, let's think about how we may try improving a tour.|

In [68]:
G = vl.grid_instance(4,4)
tour = [6,9,8,12,13,14,15,11,10,5,4,0,1,2,3,7,6]
show(vl.tour_plot(G, tour, width=300, height=300))

**Q12:** Look at the tour above. How might you improve this tour? How could you generalize this strategy?

**A:** <font color='blue'> There are two edges that cross and we could switch them around! We can generalize this by finding edges that could improve by a switch and making switches until there are no more switches. </font>

We will examine a tour improvement heuristic called 2-OPT in this part. 2-OPT looks for pairs of edges which can be reconnected to strictly improve the tour cost. (Note: there is only one way to reconnect a pair of edges). It continues in this fashion until no more improvements can be made. Let's run 2-OPT on our example from **Q12**! We will use `plot_two_opt` to  generate a visualization of 2-OPT. In each iteration, 2 edges will be highlighted red and 2 will be highlighted blue. The red edges indicate the current position and the blue indicate the positon they will be reconnected in.

In [69]:
show(vl.tsp_heuristic_plot(G, '2-OPT', tour=tour, width=300, height=300))

Now, we will run 2-OPT after the nearest neighbor heuristic on a 5x5 (euclidian distance) example.

In [70]:
# First, we run the nearest neighbor heuristic to get an initial tour
G = vl.grid_instance(5,5, manhattan=False)
tour = vl.nearest_neighbor(G)
show(vl.tour_plot(G, tour))

In [71]:
show(vl.tsp_heuristic_plot(G, '2-OPT', tour=list(tour)))

**Q13:** Run 2-OPT a few times. Do you get the same result every time? Why or why not?

**A:** <font color='blue'> Yes. There is no randomness in this algorithm.</font>

Let's run 2-OPT a few times on a slightly larger grid.

In [72]:
G = vl.grid_instance(9,9, manhattan=False)
tour = vl.nearest_neighbor(G)
show(vl.tsp_heuristic_plot(G, '2-OPT', tour=list(tour)))

**Q14:** After running 2-OPT, do you ever get a tour which crosses itself? When using euclidian distances, is this even possible? Explain why or why not.

**A:** <font color='blue'> No. This is not possible. Assume you have a tour that crosses itself. Consider the two edges that cross. You can always reconnect them to be cheaper. Hence, 2-OPT has not been run to completion. It follows that 2-OPT always terminates with a tour with no crosses. </font>

Let's compare the heuristics with and without executing 2-OPT. While we are at it, let's compare to the optimal solution which has already been computed.

In [73]:
G = vl.grid_instance(6,6, manhattan=False)
tour = optimal_tour('6x6_grid')
optimal_cost = vl.tour_cost(G, tour)
show(vl.tour_plot(G, tour))

In [74]:
n = 50
nearest_neighbor_total = 0
nearest_insertion_total = 0
furthest_insertion_total = 0
nearest_neighbor_2_total = 0
nearest_insertion_2_total = 0
furthest_insertion_2_total = 0
optimal_total = 0
for i in range(n):
    nearest_neighbor_total += vl.tour_cost(G, vl.nearest_neighbor(G))
    nearest_insertion_total += vl.tour_cost(G, vl.nearest_insertion(G))
    furthest_insertion_total += vl.tour_cost(G, vl.furthest_insertion(G))
    nearest_neighbor_2_total += vl.tour_cost(G, vl.two_opt(G, vl.nearest_neighbor(G)))
    nearest_insertion_2_total += vl.tour_cost(G, vl.two_opt(G, vl.nearest_insertion(G)))
    furthest_insertion_2_total += vl.tour_cost(G, vl.two_opt(G, vl.furthest_insertion(G)))
print("Nearest Neighbor: %s" % (nearest_neighbor_total / n))
print("Nearest Neighbor + 2-OPT: %s" % (nearest_neighbor_2_total / n))
print("Nearest Insertion: %s" % (nearest_insertion_total / n))
print("Nearest Insertion + 2-OPT: %s" % (nearest_insertion_2_total / n))
print("Furthest Insertion: %s" % (furthest_insertion_total / n))
print("Furthest Insertion + 2-OPT: %s" % (furthest_insertion_2_total / n))
print("Optimal: %s" % (optimal_cost))

Nearest Neighbor: 44.80750793304242
Nearest Neighbor + 2-OPT: 37.745483603559066
Nearest Insertion: 38.39719436592539
Nearest Insertion + 2-OPT: 37.804919498407585
Furthest Insertion: 37.615301439062705
Furthest Insertion + 2-OPT: 37.7231284194721
Optimal: 36.0


**Q15:** Compare the heuristics to their before and after 2-OPT performance. Compare them to the optimal.

**A:** <font color='blue'> The nearest neighbor heuristic improved the most from 2-OPT. Both nearest and furthest insertion were often not improved by 2-OPT and came very close to the optimal. After running 2-OPT, the nearest neighbor heuristic was *near* optimal as well. </font>

For fun, let's go back to the 23 US city example. Let's run 2-OPT on the tour you created in **Part I** (or a new one if you would like). To do this, you will need to define the tour as follows:

In [75]:
G = vl.create_network(nodes, manhattan=False)
show(vl.create_tour_plot(G, width=600, height=375, image='images/us.png'))

**Q16:** Set the `tour` variable to be the tour you manually created.

In [76]:
# We can define a tour like this:
tour = [22,21,19,20,18,15,17,16,14,13,10,12,11,8,1,2,3,4,9,7,0,6,5,22]

# After manually creating a tour, you can copy the list associated with that tour from the bottom-right

# TODO: Define your tour.

### BEGIN SOLUTION
tour = [22,21,19,20,18,15,17,16,14,13,10,12,11,8,1,2,3,4,9,7,0,6,5,22]
### END SOLUTION

Run 2-OPT!

In [77]:
show(vl.tsp_heuristic_plot(G, '2-OPT', tour=list(tour), width=600, height=375, image='images/us.png'))

**Q17:** Did 2-OPT improve your tour? By how much?

**A:** <font color='blue'> (Based on example tour) Yes. It went from 9592.0 to 8356.8. </font>

Now, let's look at an optimal solution!

In [78]:
tour = optimal_tour('us_cities_23')
show(vl.tour_plot(G, tour, width=600, height=375, image='images/us.png'))

**Q18:** Was your tour optimal before or after 2-OPT?

**A:** <font color='blue'> (Based on example tour) It was not optimal before 2-OPT but it become optimal afterwards! </font>

## Part IV: TSP Application: PCB Drilling

In the previous parts of this lab, we have considered the original application of the TSP to touring a set of cities. We also considered an arbitrary grid of nodes. In this part, we will consider a new problem to which the TSP can be applied.

Consider the manufacturing of printed circuit boards (PCBs). PCBs are used to mount integrated cicuits and combine them with other hardware. Here is a picture of a PCB after the hardware has been mounted.

<img src="images/pcb.jpg" width="500" height="380" />

*Taken from [Wikipedia](https://en.wikipedia.org/wiki/Printed_circuit_board)*

Before the hardware can be mounted, a large number of small holes must be drilled in the board. The hardware is the mounted by placing the pins of the component in these holes. Below, you can see the small holes before the hardware is mounted.

<img src="images/pcb_holes.jpg" width="444" height="250" />

*Taken from [AiPCBA](https://www.aipcba.com/pcb/pcb-boards.html)*

We are now left with an optimization problem! We have a set of holes that must be drilled in a PCB. We must specify an order to drill these holes to our drilling machine. Our goal is to choose an order that minimizes the total distance the drill travels. This problem is analogous to the TSP!

**Q19:** What are the "nodes" in the PCB dilling problem?

**A:** <font color='blue'> The holes that must be drilled.</font>

**Q20:** What is the distance between two "nodes" in the PCB dilling problem? (Assume the drill can travel in a straight line from one hole to the next).

**A:** <font color='blue'> The euclidian distance between the two holes.</font>

**Q21:** In answering **Q18** and **Q19**, you have fully described an input to the TSP. Assume we have a way of solving the TSP. How would you interpret the solution to the TSP (a tour of the nodes) as a solution to the PCB drilling problem?

**A:** <font color='blue'>The tour of nodes is equivalent to an ordered list of holes to drill.</font>

**Q22:** Why is the optimal TSP tour correspond to an optimal PCB drilling solution?

**A:** <font color='blue'>The optimal TSP tour minimizes the distance traveled between the nodes which is equivalent to the distance traveled between the holes to be drilled. This is exactly what we wish to minimize in the PCB drilling problem.</font>

Let's start by looking at a small PCB drilling instance: `xqf131`.

In [79]:
nodes = pd.read_csv('data/xqf131.csv', index_col=0)
nodes.head() # .head() restricts the display to just show the first 5 rows.

,name,x,y
0,1,0.0,13.0
1,2,0.0,26.0
2,3,0.0,27.0
3,4,0.0,39.0
4,5,2.0,0.0


In [80]:
# Like before, we need to construct a graph from this dataframe of nodes
G = vl.create_network(nodes)

Let's apply some of our TSP heuristics and 2-OPT to get some good feasible solutions to this problem!

In [81]:
# Nearest neighbor
tour = vl.nearest_neighbor(G)
show(vl.tour_plot(G, tour))

In [82]:
# Improve with 2-OPT
show(vl.tsp_heuristic_plot(G, '2-OPT', tour=list(tour)))

In [83]:
# Nearest insertion
tour = vl.nearest_insertion(G)
show(vl.tour_plot(G, tour))

In [84]:
# Optimal
tour = optimal_tour('xqf131')
show(vl.tour_plot(G, tour))

**Q23:** What were the following tour costs: nearest neighbor, nearest neighbor + 2-OPT, nearest insertion, and optimal? Which heuristic was closest to the optimal?

**A:** <font color='blue'>Will vary. I had 713.9, 608.3, 661.2, and 567.1. Nearest neighbor + 2-OPT  was the closest to optimal. </font>

Lastly, let's look at a well cited PCB instance in TSP literature: `pcb442`.

In [85]:
nodes = pd.read_csv('data/pcb442.csv', index_col=0)
G = vl.create_network(nodes)

In [86]:
# Furthest insertion
tour = vl.furthest_insertion(G)
show(vl.tour_plot(G, tour))

In [87]:
# Optimal
tour = optimal_tour('pcb442')
show(vl.tour_plot(G, tour))

## Part V: TSP Application: PCB Etching

In this final part, we will continue to think about TSP applications arising in PCB manufacturing. We will introduce two added complications. 

First, in **Part IV**, we ignored a crucial part of the problem: where does the drill bit start? The drill bit must start and end in the same location. It is not good enough to just provide a tour of the holes to be drilled.

**Q24:** If we just consider a tour of the holes, we miss two distances that must be travelled. What are they?

**A:** <font color='blue'>The distance from the start position to the first hole drilled and the distance from the last hole drilled back to the start position.</font>

Let's assume the start position of the drill is in the bottom left at $(0,0)$ and all of the holes to be drilled are at positions $(x,y)$ for $x,y > 0$

**Q25:** How can we adjust our TSP input to account for this? (Hint: how do the set of nodes change and what are the distances between them?)

**A:** <font color='blue'>We introduce an additional node for the start position. The distances between the start position and holes is still just the euclidian distance.</font>

If you look back at `pcb442`, you can see that the bottom left node actually represents the drill start location not a hole that needs to be drilled!

Let's add the final complication. Suppose, instead of drilling holes, we need to make a series of etchings in the boards. Each etching has a starting location and an ending location. Hence, once the etching machine arrives at a starting location, it must make the etching and go to the corresponding ending location immediately. Below, we have an example of an etching instance. The bottom-left node represents the starting location for the etching machine. Each etching segment has a blue start location and a red end location.

In [88]:
nodes = pd.read_csv('data/xqf131_etching.csv', index_col=0)
G = vl.create_network(nodes, directed=True, manhattan=False, x_i='x_start', y_i='y_start', x_j='x_end', y_j='y_end')
show(etching_tour_plot(G, [], width=600))

**Q26:** What are the "nodes" in the PCB etching problem? (Hint: Don't forget about the starting location of the etching machine.)

**A:** <font color='blue'> Each etching (a combination of a starting and ending location) is a node. The final node is an etching with the same starting and ending location.</font>

*Ask a TA if you are struggling with **Q25**. Once you get that, the next questions will become clearer!*

**Q27:** What is the distance between two "nodes" in the PCB etching problem? (Assume the drill can travel in a straight line between two locations).

**A:** <font color='blue'> The distance between node $i$ and node $j$ is the distance from the ending location of node $i$ to the starting location of node $j$.</font>

**Q28:** Our past TSP examples have been *symmetric* in that the distance from node $i$ to node $j$ was the same as the distance from node $j$ to node $i$. Is the TSP input for a PCB etching problem symmetric? Why or why not?

**A:** <font color='blue'> No. The distance from the end of $i$ to the start of $j$ may be different than the distance from the end of $j$ to the start of $i$</font>

Again, we find we can still use the TSP to solve this new problem! Let's start by applying some of our TSP heuristics. When printing the solution, the dashed lines indicate when the etching machine is moving between etchings. 

In [89]:
# Nearest neighbor
show(etching_tour_plot(G, vl.nearest_neighbor(G), width=600))

In [90]:
# Nearest insertion
show(etching_tour_plot(G, vl.nearest_insertion(G), width=600))

In [91]:
# Furthest insertion
show(etching_tour_plot(G, vl.furthest_insertion(G, initial_tour=[0,70,0]), width=600))

Let's see how these solutions compare to the optimal tour!

In [92]:
# Optimal
show(etching_tour_plot(G, optimal_tour('xqf131_etching'), width=600))

**Q29:** Compare the heuristics performance. How did they compare to the optimal solution?

**A:** <font color='blue'>Will vary. Nearest neighbor did 879.2, nearest insertion did 715.5, furthest insertion did 667.1, and the optimal was 591.7. Furthest insertion did the best and was within 13% of the optimal value.</font> 

You may have noticed that we did not use 2-OPT to try and improve the tours. In fact, 2-OPT only applies when the distances are symmetric. In **Q27**, you should have found that is not the case for the PCB drilling problem.

**Bonus:** Why does 2-OPT only apply when the distances are symmetric?

**A:** <font color='blue'> When we reconnect the edges, a section of the tour is now traversed in the opposite direction. When the distances are symmetric, this does not have any affect on the objective value. However, with asymmetric distances, this *does* affect the objective value!</font>